## Task2
####  Name:Bolin Yang


Date: 09/04/2019

Version: 2.0

Environment: Python 3.7.0 and Anaconda 3

Libraries used: 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.7) 
* nltk 3.2.2 (Natural Language Toolkit, included in Anaconda Python 3.7)
* nltk.collocations (for finding bigrams, included in Anaconda Python 3.7)
* nltk.tokenize (for tokenization, included in Anaconda Python 3.7)
* from itertools import chain
* from nltk.probability import *

## 1. Introduction
This assignment comprises the execution of different text pre-processing, converting the extracted data into a proper format.In this assessment, it is required to write Python code to preprocess a set of information of each unit and convert them into numerical representations (which are suitable for input into recommender-systems/ information-retrieval algorithms).After converting the pdf to text by the application called Adobe Acrobat Reader DC, I divide the entire text into sections according the unit id and store information of each unit into each text.Then, write funciton and do preprocessing and store the extracted data into specified text.

## 2. Import Libraries

In [1]:
##import library
import re
import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from itertools import chain
from nltk.probability import *
from nltk.corpus import reuters
import nltk.data
from collections import Counter

## 3. Pre-Processing Text
Write functions for some steps in pre-processing but the order of the functions is same as the process of text pre-processing.The basic steps are as follows:
* Scan pdf to text ,read text and split into separate texts
* Write function for:
  * Tokenization
  * Case normlize the first word with a capital letter in the beginning of a sentence/line.
        * Completed with regex.
  * Use nltk PorterStemmer to do stemming.
  * remove the vocab less than 3


### 3.1 Read file 

In [2]:
###use Adobe Acrobat Reader DC to transmit pdf to text
##Read text and split it by unit ,each unit and its information are stored into a text
rawtext= open("Data_file/units_pdf_totext.txt",'r')
line = rawtext.read()
lines=line.replace("Title Synopsis Outcomes",'') #delete the title we don't need 
name=re.findall("\n[A-Z]{3}[0-9]{4}\s",lines)    #extract the unitid and store them into a list 
##delete the "\n" in each unit id and store into a new list
names=[]
for item in name:
    nn=item.replace("\n",'')
    names.append(nn)

#divide the information of each unit and delete unitid
texlist=re.split(r"\n[A-Z]{3}[0-9]{4}\s",lines)
lists=texlist[1:]
#print(len(lists))
index=0
rawlists=[]  #delete the \n and \\n of the lists
for each in lists:
    eachs=re.sub("\n",'',each)
    eachs = re.sub("\\n", '',eachs)
    rawlists.append(eachs)
    ##store units' information in separate text file
    #filename="unitdata/"+ str(index) +".txt"
    #file= open(filename,"w")
    #file.write(each)
    #ret+=eachs
    #index+=1

#print(len(rawlist))
print(rawlists[1:4])
print(len(rawlists))

["This unit provides the opportunity for high-achieving students to undertake an individual research project in Genetics with an academic supervisor. It includes a critical literature review, experimental design and data analysis. The student must maintain regular contact with supervisor(s) and subject coordinator. ['Comprehend the fundamental process and requirements of scientific research;', 'Review and critically evaluate the scientific literature within a relevant discipline;', 'Demonstrate skills in experimental design, data collection and statistical analysis;', 'Interpret experimental results, and place the results in the broader context of the research discipline;', 'Communicate scientific findings and their implications, via oral presentations and written reports.'] ", "This subject explores key issues arising from the rapid change, development and growth in international communications. It examines the impact of globalisation and shifts in production, distribution and consump

### 3.2 Define functions

### 3.2.1 Word tokenization

In [4]:
##the word tokenization
def token(text):
    tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")
    #tokenizer.tokenize(text)
    return tokenizer.tokenize(text)

### 3.2.2 Get Stopword 

In [5]:
##remove the words in stopword list
def stopword_token(text):
    swordsfile = "Data_file/stopwords_en.txt"
    with open(swordsfile) as f:
        stopwords = f.read().splitlines()
        stopwords_set = set(stopwords)
        stopped_tokens = [w for w in text if w not in stopwords_set]
        return stopped_tokens


### 3.2.3 Case normalization
* token be normalized to lowercase 

In [6]:
def case_normalize(text):
    casepattern=re.compile(r"(^[A-Z]\w*|[^\w\&,\)\(;-]*\s[A-Z][a-z]+|\n\['[A-Z][a-z]+|,\s'[A-Z][a-z]+)")
    return casepattern.sub(lambda x: x.group(0).lower(), text)

### 3.5 Stemming
* use Porter stemmer
* remove the vocab less then 3

In [7]:
def proce_stem(text):
    stemmer = PorterStemmer()
    # [w, stemmer.stem(w) for w in text]
    stemed_tokens = list(map(lambda w: stemmer.stem(w) if not w[0].isupper() else w, text))

    return stemed_tokens

In [8]:
def remove_word(text):
    fintokens = list(filter(lambda w: len(w) >= 3, text))
    return fintokens

## 4. PreProcessing
The possible steps of text pre-processing are not the same as the steps of function as above.When do preprocessing, the steps as follow:
* case normalization
* word tokenization
* remove vocab in stopword list
* generate bigrams and retokenize
* stem tokens use Porter stemmer
* remove the vocab which length less than 3
* remove the rare tokens less 5% from vocab


### 4.1 Process the first three steps

In [9]:
## There are totally 200 units in the pdf however 5 units and their information are same 
## when we tramsform the unit and their information into a dict, so import counter package to check which units are 
## repeat and check whether their information are also repeat.
## After that, do pre-processing 

from collections import Counter
Counter(names)
print(Counter(names))##195units are unique

#lower the fist capital word in the sentence and tokenize,then remove stopwords
token_lists=[]
for each in rawlists:
    nor_text=case_normalize(each)
    token_list=token(nor_text)
    stopp = stopword_token(token_list)
    token_lists.append(stopp)
#print(token_lists)
print(len(token_lists))

#store the extracted data into a dict
data_raw={}
if len(names)==len(token_lists):
    for i in range(len(token_lists)):
        data_raw[names[i]]=token_lists[i]

print(len(data_raw))

#count total document number
docnum=len(data_raw)



Counter({'ATS2060 ': 2, 'MKF5403 ': 2, 'BRM5013 ': 2, 'ATS2094 ': 2, 'OHS1000 ': 2, 'GEN5010 ': 1, 'GEN3990 ': 1, 'APG5400 ': 1, 'PGW5205 ': 1, 'LAW4122 ': 1, 'NUR5022 ': 1, 'ATS2305 ': 1, 'ATS3012 ': 1, 'BFF2401 ': 1, 'LAW4158 ': 1, 'MGS3991 ': 1, 'MKF5955 ': 1, 'BTM5909 ': 1, 'CHE2166 ': 1, 'ATS3820 ': 1, 'APG5190 ': 1, 'CIV4288 ': 1, 'ATS3145 ': 1, 'VPR1012 ': 1, 'BMA1011 ': 1, 'LAW4119 ': 1, 'ATS2169 ': 1, 'EAE2011 ': 1, 'FIT1006 ': 1, 'RSE3020 ': 1, 'ATS3908 ': 1, 'MMD4001 ': 1, 'ATS3311 ': 1, 'MAE2403 ': 1, 'FIT5137 ': 1, 'MGF5940 ': 1, 'ATS1091 ': 1, 'ASP5000 ': 1, 'ATS3189 ': 1, 'ACC3600 ': 1, 'MEC4402 ': 1, 'BMS5006 ': 1, 'AHT2740 ': 1, 'MTE4598 ': 1, 'CIV2242 ': 1, 'ATS2910 ': 1, 'RAD2061 ': 1, 'MKF5917 ': 1, 'ASP3231 ': 1, 'MKF3621 ': 1, 'GEN2041 ': 1, 'CIV6883 ': 1, 'CPS5001 ': 1, 'PGW5203 ': 1, 'MBA5213 ': 1, 'PAR3031 ': 1, 'SWM5112 ': 1, 'FIT5145 ': 1, 'PSY5261 ': 1, 'AHT4111 ': 1, 'MPH5286 ': 1, 'CIV6888 ': 1, 'ASP4002 ': 1, 'ATS3867 ': 1, 'CHE3163 ': 1, 'ADA5011 ': 1, '

### 4.2 Bigrams
### 4.2.1 Generate bigrams
For method bigram_finder.apply_freq_filter(min_freq), chose min_freq = 8 can get enough 200 bigrams

In [10]:
#Generate bigrams
all_words= list(chain.from_iterable(data_raw.values()))
print(len(all_words))
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
bigram_finder.apply_freq_filter(3)          
bigram_finder.apply_word_filter(lambda w: len(w) < 3)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-200 bigrams
print(len(top_200_bigrams))

20284
200


### 4.2.2 Retokenize the *tokenizeRawData* with *MWETokenizer*
> Get *colloc_tokens* dict

In [11]:
mwetokenizer = MWETokenizer(top_200_bigrams)

tott= list(mwetokenizer.tokenize(tokens) for tokens in data_raw.values())
print(len(tott))

195


### 4.3 Remove words and stemmer

In [12]:
#pre-processing
finall_token=[]

for each in tott:
    #stopp=stopword_token(each)
    words=remove_word(each)
    stemmed_token=proce_stem(words)
    #print(stopp)
    finall_token.append(stemmed_token)

print(len(finall_token))

#store extract data into a dict
colloc_tokens={}
for i in range(len(data_raw)):
    vv=list(set(names))
    colloc_tokens[vv[i]]=finall_token[i]

195


### 4.4 Get a vocab and Remove tokens by threshold

In [13]:
highThresholdNumber =int( 0.95 * docnum)
lowThresholdNumber = int(0.05 * docnum)

#print(highThresholdNumber)
#print(lowThresholdNumber)

vocab = list(chain.from_iterable([set(value) for value in finall_token]))

print(len(vocab))


12954


> Remove tokens with document frequency less than 5% or more than 95% thresholad from the vocab.<br>
> *Get **final_List** for outpuit*

In [14]:
fq_token=nltk.FreqDist(vocab)
#print(fq_token)

for key in colloc_tokens.keys():
    colloc_tokens[key] = [w for w in colloc_tokens[key] if lowThresholdNumber < fq_token[w] < highThresholdNumber]

In [15]:
newVocab = list(chain.from_iterable([set(value) for value in colloc_tokens.values()]))

In [16]:
final_list = sorted(set(newVocab))
print(len(final_list))

303


## 6. Output for task 2
* vocab.txt​: It contains the ​bigrams and unigrams
tokens in the following format, ​token_string:integer_index.​ Words in
the vocabulary must be sorted in alphabetical order.
* countVec.txt:​ the txt file contains all the “selected”
units in the data-set. Each line in the txt file contains the sparse representations of one of the units in the data-set in the following format ​unit_code, token_index:count, token_index:count,...

In [17]:
vocabFile = "units_inform" + "_vocab.txt"
with open(vocabFile, 'w') as f:
    for i in range(len(final_list)):
        line = final_list[i] + ":" + str(i) + "\n"
        f.write(line)

In [18]:
countVecFile = "units_inform" + "_countVec.txt"
with open(countVecFile, 'w') as f:
    for key in colloc_tokens:
        lines = key
        for item in colloc_tokens[key]:
            lines = lines + ", " + str(final_list.index(item))+ ":" + str(colloc_tokens[key].count(item))
        lines = lines + "\n"
        f.write(lines)